# 02 - Data from the Web


In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need! You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this brief tutorial to understand quickly how to use it.

In [741]:
from bs4 import BeautifulSoup
import requests
import json
import string
import pandas as pd
import re
import math
import numpy as np

# Methods

## Method to sort the ranking by a specific attribute

In [742]:
def sort_df_by(df, attribute_to_sort_by, rank):
    df_by = df.sort_values([attribute_to_sort_by, rank], ascending=[True, True]);
    df_by = df_by.drop_duplicates(subset = attribute_to_sort_by);
    df_by = df_by.sort_values(rank)
    df_by = df_by.loc[:, ['University', rank, attribute_to_sort_by]]
    return  df_by

## Ratio calculation

In [743]:
def ratio(a, b):
    a = float(a)
    b = float(b)
    if b == 0:
        return a
    return ratio(b, a % b)

# Task 1 - Data from QS ranking

## Question 1

Multiple strategies have been testes in order to find the data used to construct the table of the QS Rankings. However, only one of them led to the expected result. We opened the Web Inspector on the website. Among the ressources used by the page, a text file named 'XHR/357051.txt'  was containing the expected ranking with the desired informations. 

In [744]:
URL_QS = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508497791260'
r = requests.get(URL_QS)
page_body = r.json()
qs_df = pd.DataFrame(page_body['data'])
qs_df = qs_df.drop(['core_id','logo','guide','cc', 'nid', 'stars','score'], axis=1).head(200)

Additionnal informations needed to be accessed on each university webpage. We used the 'url' column, went to the university page and extracted all the number values we wanted and placed them in additionnal columns. In some cases, there were no informations on the webpage, in this case the columns are filed with NaN.

In [745]:
# Going on each university page for each index of the table

for index_ in range(0,qs_df.shape[0]):
    
    URL_ADD = 'https://www.topuniversities.com' + qs_df.get_value(index_, 'url')
    r = requests.get(URL_ADD)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    # Request all numbers that are displayed on the page, we know their order
    all_numbers = soup.findAll('div', {'class' : 'number'})
    
    # If the page contains no numbers, we will store NaN
    if not all_numbers:
        total_faculty_number = 'NaN'
        interna_faculty_number = 'NaN'
        total_students_number = 'NaN'
        interna_students_number = 'NaN'
   
    else:    
    # Total Faculty Members
        total_faculty = all_numbers[0]
        total_faculty_number = int(re.sub("[^\\d]",'',str(total_faculty)))
    
    # International Faculty Members
        interna_faculty = all_numbers[1]
        interna_faculty_number = int(re.sub("[^\\d]",'',str(interna_faculty)))
    
    # Total Students
        total_students = all_numbers[2]
        total_students_number = int(re.sub("[^\\d]",'',str(total_students)))  
    
    # International Students
        interna_students = all_numbers[3]
        interna_students_number = int(re.sub("[^\\d]",'',str(interna_students)))
    
    # Adding the found values in the columns
    qs_df.set_value(index_, 'Faculty Members - Total', total_faculty_number)
    qs_df.set_value(index_, 'Faculty Members - International', interna_faculty_number)
    qs_df.set_value(index_, 'Nb Students - Total', total_students_number)
    qs_df.set_value(index_, 'Nb Students - International', interna_students_number)
        

In [746]:
qs_df;

In [747]:
for index, row in qs_df.iterrows():
    if np.isfinite(row['Nb Students - Total']) and np.isfinite(row['Nb Students - International']) or np.isfinite(row['Nb Students - Total']) and np.isfinite(row['Faculty Members - Total']):
        qs_df.loc[index, 'QS Ratio Int. Students'] = ratio(row['Nb Students - Total'], row['Nb Students - International'])
        qs_df.loc[index, 'QS Ratio Student/Staff'] = ratio( row['Nb Students - Total'],row['Faculty Members - Total'])


In [748]:
qs_df = qs_df.loc[ :, ['title', 'rank_display', 'country', 'region', 'QS Ratio Int. Students', 'QS Ratio Student/Staff' ]]


In [749]:
qs_df = qs_df.rename(index=str, columns={ "title" : "University", "rank_display": "QS rank",
                                  "No Students - Total" : "QS Total Students",
                                  "No Students - International" : "QS International Students",
                                  "Faculty Members - Total" : "QS Total Faculty Members",
                                  "Faculty Members - International" : "QS International Faculty Members"
                                 });

In [750]:
qs_df;

### By country

In [751]:
qs_country = sort_df_by(qs_df, 'country', 'QS rank')
qs_country

,University,QS rank,country
0,Massachusetts Institute of Technology (MIT),1,United States
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Switzerland
100,University of Leeds,101,United Kingdom
103,Eindhoven University of Technology,104,Netherlands
105,Yonsei University,106,South Korea
106,"KIT, Karlsruhe Institute of Technology",107,Germany
10,"Nanyang Technological University, Singapore (NTU)",11,Singapore
111,Uppsala University,112,Sweden
118,Aarhus University,119,Denmark
120,Universidade de São Paulo,121,Brazil


### By Region

In [646]:
qs_region = sort_df_by(qs_df, 'region', 'QS rank')
qs_region

,University,QS rank,region
0,Massachusetts Institute of Technology (MIT),1,North America
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Europe
105,Yonsei University,106,Asia
120,Universidade de São Paulo,121,Latin America
150,University of Otago,151,Oceania
190,University of Cape Town,191,Africa


## Data from Times Higher Education ranking

In [755]:
URL_THE = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL_THE)
page_body = r.json()
the_df = pd.DataFrame(page_body['data'])

### Clean the data

In [756]:
the_df = the_df[:200]

## Quelles données nous sont inutiles? 

Je propose de garder: 
- name
- location
- rank
- stats number students
- stats % intl students (to rename and maybe change the percentage by a number)
- stat student staff ratio


#### We calculate the number of International students and then the ratio

In [757]:
# Firts we need to remove the '%' symbol from the column "THE % International Students"
# Then we convert the string to int and we do the calculation

In [758]:
the_df['stats_number_students'] = pd.to_numeric(the_df['stats_number_students'].str.replace(',',''))
the_df['stats_pc_intl_students'] = pd.to_numeric(the_df['stats_pc_intl_students'].str.replace('%',''))
for index, row in the_df.iterrows():
    the_df.loc[index, 'stats_pc_intl_students'] = round(row['stats_number_students']*row['stats_pc_intl_students']*0.01)
    #if np.isfinite(row['stats_number_students']) and np.isfinite(row['Nb Students - International']) or np.isfinite(row['Nb Students - Total']) and np.isfinite(row['Faculty Members - Total']):
    the_df.loc[index, 'THE Ratio Int. Students'] = ratio(row['stats_number_students'], row['stats_pc_intl_students'])


In [759]:
the_df = the_df.loc[ :, ['name', 'rank', 'location','THE Ratio Int. Students', 'stats_student_staff_ratio']]


#### We rename the Data in order to combine the two DF later

In [760]:
the_df = the_df.rename(index=str, columns={ "name" : "University",
                                  "rank": "THE rank", "location": "country",
                                  "stats_student_staff_ratio" : "THE Ratio Students/Staff"
                                 });

In [762]:
the_df;

### By Country

In [754]:
the_country = sort_df_by(the_df, 'country', 'THE rank')
the_country

,University,THE rank,country
0,University of Oxford,1,United Kingdom
102,Maastricht University,103,Netherlands
106,Ghent University,107,Belgium
107,University of Montreal,108,Canada
114,École Polytechnique,115,France
115,Fudan University,116,China
11,Yale University,12,United States
124,University of Mannheim,125,Germany
135,University of Zurich,136,Switzerland
139,Pompeu Fabra University,140,Spain


### By Student Ratio

In [ ]:
the_students_ratio = sort_df_by(the_df, 'country', 'THE rank')
the_students_ratio

# On met les deux ensemble

In [605]:
result = pd.merge(qs_df, the_df, how='outer', on='University')

In [606]:
result

,University,QS rank,country_x,region,QS Total Students,QS International Students,QS Total Faculty Members,QS International Faculty Members,THE rank,country_y,THE Ration Male/Female,THE Total Students,THE International Students,THE Ratio Students/Staff
0,Massachusetts Institute of Technology (MIT),1,United States,North America,11067.0,3717.0,2982.0,1679.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Stanford University,2,United States,North America,15878.0,3611.0,4285.0,2042.0,=3,United States,42 : 58,15845.0,3486.0,7.5
2,Harvard University,3,United States,North America,22429.0,5266.0,4350.0,1311.0,6,United States,None,20326.0,5285.0,8.9
3,California Institute of Technology (Caltech),4,United States,North America,2255.0,647.0,953.0,350.0,NaN,NaN,NaN,NaN,NaN,NaN
4,University of Cambridge,5,United Kingdom,Europe,18770.0,6699.0,5490.0,2278.0,2,United Kingdom,45 : 55,18389.0,6436.0,10.9
5,University of Oxford,6,United Kingdom,Europe,19720.0,7353.0,6750.0,2964.0,1,United Kingdom,46 : 54,20409.0,7755.0,11.2
6,UCL (University College London),7,United Kingdom,Europe,31080.0,14854.0,6345.0,2554.0,NaN,NaN,NaN,NaN,NaN,NaN
7,Imperial College London,8,United Kingdom,Europe,16090.0,8746.0,3930.0,2071.0,8,United Kingdom,37 : 63,15857.0,8721.0,11.4
8,University of Chicago,9,United States,North America,13557.0,3379.0,2449.0,635.0,9,United States,44 : 56,13525.0,3381.0,6.2
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Switzerland,Europe,19815.0,7563.0,2477.0,1886.0,NaN,NaN,NaN,NaN,NaN,NaN
